In [15]:
import numpy as np
import pandas as pd
pd.set_option('display.width',1000)
from pydotplus import graphviz
from PIL import Image

In [77]:
dataset_org = pd.read_csv(r'data\watermelon_2.csv', index_col = 0)
TrueLabel = '是'
test_idx = [3,4,7,8,10,11,12]
train_idx = list(set(list(range(17))) - set(test_idx))
train = dataset_org.iloc[train_idx]
test = dataset_org.iloc[test_idx]
print(train)
print(test)

    色泽  根蒂  敲声  纹理  脐部  触感 好瓜
编号                           
1   青绿  蜷缩  浊响  清晰  凹陷  硬滑  是
2   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  是
3   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  是
6   青绿  稍蜷  浊响  清晰  稍凹  软粘  是
7   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  是
10  青绿  硬挺  清脆  清晰  平坦  软粘  否
14  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  否
15  乌黑  稍蜷  浊响  清晰  稍凹  软粘  否
16  浅白  蜷缩  浊响  模糊  平坦  硬滑  否
17  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  否
    色泽  根蒂  敲声  纹理  脐部  触感 好瓜
编号                           
4   青绿  蜷缩  沉闷  清晰  凹陷  硬滑  是
5   浅白  蜷缩  浊响  清晰  凹陷  硬滑  是
8   乌黑  稍蜷  浊响  清晰  稍凹  硬滑  是
9   乌黑  稍蜷  沉闷  稍糊  稍凹  硬滑  否
11  浅白  硬挺  清脆  模糊  平坦  硬滑  否
12  浅白  蜷缩  浊响  模糊  平坦  软粘  否
13  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  否


In [78]:
class Node(object):
    def __init__(self, attr_init = None, label_init = None, attr_down_init=None):
        '''
            definition of decision node class
            
        @param attr_init:     attribution as parent for a new branching
        @param attr_down:     dict {key, val}
                        key:    categoric: categoric attr_value
                                continuous: <= div_val for small part
                                            >  div_val for big part
                        value:  children
        @param label    :     class label (the majority of current sample labels)
        
        '''
        self.attr = attr_init
        self.label = label_init
        if attr_down_init is None:
            self.attr_down = dict()
        else:
            self.attr_down = attr_down_init
    def __str__(self):
        if self.attr is None:
            ret = 'Label = %s'%(str(self.label))
        else:
            ret = 'Label = %s, Attr = %s'%(str(self.label), self.attr)
        
        ret += str(self.attr_down)
        return ret

In [79]:
def Counter(array):
    '''
        calculating the appeared element and it's counts
        @param the list of element 
        @return: count : dict, the appeared element and it's counts
    
    '''
    count = dict()
    for e in array:
        count[e] = count.get(e, 0) + 1
    return count

In [109]:
def getLabelGini(df):
    labelCount = Counter(df[df.columns[-1]])
    e = 0; n = len(df)

    for val in labelCount.values():
        e += (val/n)**2
    return 1 - e
print(train)
getLabelGini(train)

    色泽  根蒂  敲声  纹理  脐部  触感 好瓜
编号                           
1   青绿  蜷缩  浊响  清晰  凹陷  硬滑  是
2   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  是
3   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  是
6   青绿  稍蜷  浊响  清晰  稍凹  软粘  是
7   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  是
10  青绿  硬挺  清脆  清晰  平坦  软粘  否
14  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  否
15  乌黑  稍蜷  浊响  清晰  稍凹  软粘  否
16  浅白  蜷缩  浊响  模糊  平坦  硬滑  否
17  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  否


0.5

In [110]:
def __gini_index(samedAttr_list, TrueLabel = TrueLabel):
    
    #print(samedAttr_list)
    numAttr_list = [len(attr) for attr in samedAttr_list]
    nSample = sum(numAttr_list)
    ent = 0
    for attr, num in zip(samedAttr_list, numAttr_list):
        trueCount = sum(attr[attr.columns[-1]] == TrueLabel)
        p = trueCount / num
        not_p = 1-p
        ent += num/nSample * (1 - p * p - not_p * not_p)
    return ent

In [119]:
#info_gain = InfoEnt(df.values[:,-1])  # info_gain for the whole label

def gain_index(df, index):
    '''
    calculating the information gain of an attribution
    
    @param df:        dataframe, the pandas dataframe of the dataset
    @param index:     the target attribution index in df
    @return info_gain:the information gain of the current classify
    @return div_val  :only valid in continuous attribution , return a real num for binary-division threshold
                      for discrete attribution , return 0
    '''
    
    nSample = len(df)
    if df[index].dtype not in (float, int):

        AttrCount = Counter(df[index])
        vals = list(AttrCount.keys())
        #print(AttrCount)
        samedAttr_list = [ df[df[index] == v] for v in vals]
        return __gini_index(samedAttr_list), 0, vals, samedAttr_list
    else:
        attr_val = sorted(list(df[index]))
        Ta = [(i+j)/2 for i,j in zip(attr_val[:-1], attr_val[1:])]
        best_threshold = 0
        best_index = 0
        best_samedAttr_list = None
        for threshold in Ta:
            small = df[df[index] <= threshold]
            big = df[df[index] > threshold]
            AttrCount = [len(small), len(big)]
            #samedAttr_list = [small, big]
            gini = __gini_index(samedAttr_list)
            if gain_t >= best_gain:
                best_index = gini
                best_threshold = threshold
                best_samedAttr_list = samedAttr_list
        vals = ['<= %.3f'%(best_threshold), '> %.3f'%(best_threshold)]
        return best_index, best_threshold, vals, best_samedAttr_list
            
for attr in train.columns[:-1]:
    gain, div_val, _,_ = gain_index(train,attr)
    print("The Gain of %3s is %.3f and best threshold is %.3f"%(attr, gain, div_val))

The Gain of  色泽 is 0.350 and best threshold is 0.000
The Gain of  根蒂 is 0.440 and best threshold is 0.000
The Gain of  敲声 is 0.400 and best threshold is 0.000
The Gain of  纹理 is 0.400 and best threshold is 0.000
The Gain of  脐部 is 0.350 and best threshold is 0.000
The Gain of  触感 is 0.500 and best threshold is 0.000


In [126]:
def optAttr_Gini(df, usedAttr = None):
    gini_index, div =0x7FFFFFFF, 0
    opt_attr = attr_val_list = val_list = None
    for attr in df.columns[:-1]:
        if usedAttr and attr in usedAttr: continue
        tmp_index, tmp_div, tmp_vals, tmp_blocks = gain_index(df,attr)
        if tmp_index <= gini_index:
            gini_index = tmp_index
            opt_attr, div = attr, tmp_div
            attr_val_list, val_list = tmp_vals, tmp_blocks
    return opt_attr, div, attr_val_list, val_list

optAttr_Gini(train)

('脐部', 0, ['凹陷', '稍凹', '平坦'], [    色泽  根蒂  敲声  纹理  脐部  触感 好瓜
  编号                           
  1   青绿  蜷缩  浊响  清晰  凹陷  硬滑  是
  2   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  是
  3   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  是
  14  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  否,     色泽  根蒂  敲声  纹理  脐部  触感 好瓜
  编号                           
  6   青绿  稍蜷  浊响  清晰  稍凹  软粘  是
  7   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  是
  15  乌黑  稍蜷  浊响  清晰  稍凹  软粘  否
  17  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  否,     色泽  根蒂  敲声  纹理  脐部  触感 好瓜
  编号                           
  10  青绿  硬挺  清脆  清晰  平坦  软粘  否
  16  浅白  蜷缩  浊响  模糊  平坦  硬滑  否])

In [128]:
def TreeGenerate(df, usedAttr=set()):
    '''
    Branching for decision tree using recursion
    @param  df  : the pandas dataframe of the dataset
    @return root: Node, the root node of decision tree
    '''
    #generating a ew root node
    new_node = Node()

    label_arr = df[df.columns[-1]]
    
    label_count = Counter(label_arr)
    node_label = sorted(label_count.items(), key = lambda x:x[1], reverse=True)[0][0]
    if label_count:
        # end if there is onlly 1 class in current node data
        # end if attriubtion array is empty
        new_node.label = node_label
        if len(label_count) <= 1:
            
            return new_node
        
        #get the optimal attriubtion for a new branching
        new_node.attr, div, attr_val_list, val_list = optAttr_Gini(df, usedAttr)
        
        if new_node.attr is None:
            return new_node
        
        if div_val != 0 :
            print("The attribution of node is %s and best threshold is %.3f"%(new_node.attr, div_val))
        else:
            print("The attribution of node is %s "%(new_node.attr))


        #recursion
        print(attr_val_list)
        
        usedAttr.add(new_node.attr)
        print("used",usedAttr)
        
        for i in range(len(attr_val_list)):
            attr = attr_val_list[i]
            data = val_list[i]
            new_node.attr_down[attr] = TreeGenerate(data)
            
        #usedAttr.discard(new_node.attr)
        #print(new_node.attr_down.keys())
    return new_node

myDT = TreeGenerate(train)

The attribution of node is 脐部 
['凹陷', '稍凹', '平坦']
used {'脐部'}
The attribution of node is 纹理 
['清晰', '稍糊']
used {'脐部', '纹理'}
The attribution of node is 触感 
['软粘', '硬滑']
used {'脐部', '触感', '纹理'}
The attribution of node is 色泽 
['青绿', '乌黑']
used {'脐部', '触感', '纹理', '色泽'}
The attribution of node is 敲声 
['浊响']
used {'敲声', '色泽', '脐部', '触感', '纹理'}
The attribution of node is 根蒂 
['稍蜷']
used {'敲声', '色泽', '脐部', '触感', '根蒂', '纹理'}


In [129]:
def TreeToGraph(i, g, root):
    '''
    build a graph from root on
    @param i: node number in this tree
    @param g: pydotplus.graphviz.Dot() object
    @param root: the root node
    
    @return i: node number after modified  
#     @return g: pydotplus.graphviz.Dot() object after modified
    @return g_node: the current root node in graphviz
    '''
    try:
        from pydotplus import graphviz
    except ImportError:
        print("module pydotplus.graphviz not found")

    if root.attr == None:
        g_node_label = "Node:%d\n好瓜:%s" % (i, root.label)
    else:
        g_node_label = "Node:%d\n好瓜:%s\n属性:%s" % (i, root.label, root.attr)
    g_node = i
    g.add_node( graphviz.Node( g_node, label = g_node_label, fontname="Microsoft Yahei" ) )
    
    for value in list(root.attr_down):
        i, g_child = TreeToGraph(i+1, g, root.attr_down[value])
        g.add_edge( graphviz.Edge(g_node, g_child, label = value, fontname="Microsoft Yahei") ) 

    return i, g_node

def DrawPNG(root, out_file):
    '''
    visualization of decision tree from root.
    @param root: Node, the root node for tree.
    @param out_file: str, name and path of output file
    '''
    try:
        from pydotplus import graphviz
    except ImportError:
        print("module pydotplus.graphviz not found")
        
    g = graphviz.Dot()  # generation of new dot   

    TreeToGraph(0, g, root)
    g2 = graphviz.graph_from_dot_data( g.to_string() )
    
    g2.write_png(out_file) 
    
path_png = 'myCART1.jpg'
DrawPNG(myDT, path_png)
im = Image.open(path_png)
#im.show()

![img](./myCART1.jpg)

In [130]:
def Predict(root, df_sample):
    while root.attr != None:
        sample_val = df_sample[root.attr]
        #print("DT attr = %s, sample val = %s"%(root.attr, str(sample_val)))
        if type(sample_val) == str:
            if sample_val in root.attr_down:
                root = root.attr_down[sample_val]
            else:
                print("The key of sample ",sample_val,"%s isn't belone to attribute %s"%(str(type(sample_val)), root.attr))
                break
        else:
            key = list(root.attr_down.keys())
            try:
                div_val = float(re.findall(r"\d+.?\d*",key[0])[0])
            except:
                print("the type of sample is unmatch to the DT attr")
                break
            if sample_val <= div_val:
                root = root.attr_down[key[0]]
            else:
                root = root.attr_down[key[1]]
    return root.label
t = train.iloc[4]
print(t)
Predict(myDT, t)

色泽    乌黑
根蒂    稍蜷
敲声    浊响
纹理    稍糊
脐部    稍凹
触感    软粘
好瓜     是
Name: 7, dtype: object


'是'

In [131]:
def PredictAccuracy(root, df_test):
    n = len(df_test)
    count = 0
    for i in range (n):
        sample = df_test.iloc[i]
        if sample['好瓜'] == Predict(myDT, sample):
            count += 1
        print(sample['好瓜'], Predict(myDT, sample))
    return count / n

PredictAccuracy(myDT, test)

是 是
是 是
是 否
否 否
否 否
否 否
否 否


0.8571428571428571

In [50]:
def preCutTreeGenerate(df, usedAttr=set(), test_acc = 0):
    '''
    Branching for decision tree using recursion
    @param  df  : the pandas dataframe of the dataset
    @return root: Node, the root node of decision tree
    '''
    #generating a ew root node
    new_node = Node()

    label_arr = df[df.columns[-1]]
    
    label_count = Counter(label_arr)
    node_label = sorted(label_count.items(), key = lambda x:x[1], reverse=True)[0][0]
    if label_count:
        # end if there is onlly 1 class in current node data
        # end if attriubtion array is empty
        new_node.label = node_label
        if len(label_count) <= 1:
            
            return new_node
        
        #get the optimal attriubtion for a new branching
        new_node.attr, div_val, div_res = optAttr_Gini(df, usedAttr)
        #------full the sub_attr for predicting in test data----
        for res in div_res:
            sub_label_count = Counter(res)
            sub_node_label = sorted(sub_label_count.items(), key = lambda x:x[1], reverse=True)[0][0]
            tmp = Node()
            new_node.attr_down = 
        #-------------------------------------------------------
        
        if new_node.attr is None:
            return new_node
        
        if div_val != 0 :
            print("The attribution of node is %s and best threshold is %.3f"%(new_node.attr, div_val))
        else:
            print("The attribution of node is %s "%(new_node.attr))
        print(div_res)
        attr_list = None
        #recursion
        if div_val == 0: #categoric variable
            attr_list = [df[new_node.attr].head(1).tolist()[0] for df in div_res]
        elif len(div_res) != 1:
            attr_list = ['<= %.3f'%(div_val), '> %.3f'%(div_val)]
        print(attr_list)
        usedAttr.add(new_node.attr)
        print("used",usedAttr)
        for i in range(len(attr_list)):
            attr = attr_list[i]
            data = div_res[i]
            new_node.attr_down[attr] = TreeGenerate(data)
            
        usedAttr.discard(new_node.attr)
        #print(new_node.attr_down.keys())
    return new_node

myDT = TreeGenerate(train)

,色泽,根蒂,敲声,纹理,脐部,触感,好瓜
编号,,,,,,,
5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,是
6,青绿,稍蜷,浊响,清晰,稍凹,软粘,是
9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,否
10,青绿,硬挺,清脆,清晰,平坦,软粘,否
12,浅白,蜷缩,浊响,模糊,平坦,软粘,否
13,青绿,稍蜷,浊响,稍糊,凹陷,硬滑,否
14,浅白,稍蜷,沉闷,稍糊,凹陷,硬滑,否
